In [1]:
from pathlib import Path

import pandas as pd
import numpy as np

import argparse

from boostedhh import utils

In [2]:
# automatically reloads imported files on edits
%load_ext autoreload
%autoreload 2

In [3]:
def csv_to_latex_efficiency(
    csv_path: str,
    signal=str,
    year=str,
    channel=str,
    output_path: str = None,
    caption: str = None,
    label: str = None,
) -> str:
    """Convert a CSV file to LaTeX table format.

    Args:
        csv_path: Path to the CSV file
        output_path: Path to save the LaTeX output (optional)
        caption: Table caption (optional)
        label: Table label for referencing (optional)

    Returns:
        LaTeX table code as a string
    """

    df = pd.read_csv(csv_path)

    latex_table = "\\begin{table}[htp]\n\\centering\n"

    if caption:
        latex_table += f"\\caption{{{caption}}}\n"

    # Add label if provided
    if label:
        latex_table += f"\\label{{{label}}}\n"

    # Generate table content
    latex_table += "\\begin{tabular}{" + "c" * len(df.columns) + "}\n"
    latex_table += "\\hline\n"

    header_titles = {
        "bbtt": r"ggF HHbb$\tau\tau$",
        "vbfbbtt": r"VBF HHbb$\tau\tau$ SM",
        "vbfbbtt-k2v0": r"VBF HHbb$\tau\tau$ $\kappa_{2V}=0$",
    }

    # Add headers
    latex_table += (
        header_titles[signal]
        + f", {year}, {channel} & "
        + " & ".join(str(val).replace("Trigger Efficiency", "") for val in df.columns[1:])
        + " \\\\\n"
    )
    latex_table += "\\hline\n"

    # Add data rows
    for _, row in df.iterrows():
        latex_table += (
            " & ".join(
                [
                    str(row.iloc[0])
                    #   .replace(", XbbvsQCD > 0.95", " QCD cut")
                    #   .replace(", XbbvsQCDTop > 0.95", " QCDTop cut")
                    .replace(">", "$>$")
                    .replace("boosted", "")
                    .replace(" (", ", $p_T $ (")
                    .replace(")", ") GeV")
                    #   .replace("(> 250)","")
                    #   .replace("(>250, >200)","")
                ]
                + [(str(val) + "\%") for val in row.iloc[1:]]
            )
            + " \\\\\n"
        )

    latex_table += "\\hline\n"
    latex_table += "\\end{tabular}\n"
    latex_table += "\\end{table}\n\n"

    # Save to file if output path provided
    if output_path:
        with open(output_path, "w") as f:
            f.write(latex_table)

    return latex_table

In [ ]:
# N-1:
for signal in ["bbtt", "vbfbbtt", "vbfbbtt-k2v0"]:
    # print("***",signal,"***")
    for year in ["2022", "2022EE", "2023", "2023BPix"]:
        print(r"\begin{landscape}")
        for channel in ["hh", "hm", "he"]:
            print(
                csv_to_latex_efficiency(
                    f"/home/users/lumori/bbtautau/plots/TriggerStudy/25Apr16/{year}/{signal}/trig_effs_{channel}.csv",
                    signal,
                    year,
                    channel,
                )
            )
        print(r"\end{landscape}")

In [5]:
def csv_to_latex_successive_removal(
    csv_path: str, output_path: str = None, caption: str = None, label: str = None
) -> str:
    """Convert a CSV file to LaTeX table format.

    Args:
        csv_path: Path to the CSV file
        output_path: Path to save the LaTeX output (optional)
        caption: Table caption (optional)
        label: Table label for referencing (optional)

    Returns:
        LaTeX table code as a string
    """

    df = pd.read_csv(csv_path)

    latex_table = "\\begin{table}[H]\n\\centering\n"

    if caption:
        latex_table += f"\\caption{{{caption}}}\n"

    # Add label if provided
    if label:
        latex_table += f"\\label{{{label}}}\n"

    # Generate table content
    latex_table += "\\begin{tabular}{" + "c" * len(df.columns) + "}\n"
    latex_table += "\\hline\n"

    # Add headers
    latex_table += (
        " & ".join(
            str(val)
            .replace("_", "\_")
            .replace("(", "")
            .replace(")", "")
            .replace("\_{", "_{")
            .replace("Trigger Efficiency", "")
            for val in df.columns
        )
        + " \\\\\n"
    )
    latex_table += "\\hline\n"

    # Add data rows
    for _, row in df.iterrows():
        latex_table += (
            " & ".join(
                str(val).replace("%", "\%")
                #   .replace(", XbbvsQCD > 0.95", "QCD cut")
                #   .replace(", XbbvsQCDTop > 0.95", "QCDTop cut")
                #   .replace("(> 250)","")
                #   .replace("(>250, >200)","")
                .replace(">", "$>$")
                .replace("boosted", "")
                .replace(" (", ", $p_T $ (")
                .replace(")", ") GeV")
                for val in row
            )
            + " \\\\\n"
        )

    latex_table += "\\hline\n"
    latex_table += "\\end{tabular}\n"
    latex_table += "\\end{table}\n\n"

    # Save to file if output path provided
    if output_path:
        with open(output_path, "w") as f:
            f.write(latex_table)

    return latex_table

In [ ]:
# Successive subtraction:
for signal in ["bbtt", "vbfbbtt", "vbfbbtt-k2v0"]:
    # print("***",signal,"***")
    for year in ["2022", "2023BPix"]:
        print(r"\begin{landscape}")
        for channel in ["hh", "hm", "he"]:
            # print("***",channel,"***")
            print(
                csv_to_latex_successive_removal(
                    f"/home/users/lumori/bbtautau/plots/TriggerStudy/25Apr16/{year}/{signal}/progressive_removal_{channel}_.csv"
                )
            )
        print(r"\end{landscape}")

In [9]:
# N-1 plots


def plot_latex_misc(input_dict=dict[str, list[str]]) -> str:
    """
    the lists are len 4 to pack in a 2x2 plot
    """

    signal = input_dict["signal"]
    year = input_dict["year"]
    channel = input_dict["channel"]
    fom = input_dict["fom"]
    isminus = input_dict["isminus"]

    # caption = ""
    paths = [
        f"figures/triggers/{year}/{signal}/{fom}_{channel}{'_'+isminus if isminus else ''}.pdf"
        for year, signal, channel, fom, isminus in zip(year, signal, channel, fom, isminus)
    ]

    latex_text = (
        r"""
    \begin{figure}[htbp]
        \centering
        \begin{tabular}{cc}
            \includegraphics[width=0.49\textwidth]{"""
        + paths[0]
        + r"""} &
            \includegraphics[width=0.49\textwidth]{"""
        + paths[1]
        + r"""} \\[1ex]
            \includegraphics[width=0.49\textwidth]{"""
        + paths[2]
        + r"""} &
            \includegraphics[width=0.49\textwidth]{"""
        + paths[3]
        + r"""}
            \label{fig}
        \end{tabular}
    \end{figure} 
    """
    )

    return latex_text


def plot_latex_yearsgrouped(signal=str, channel=str, fom=str) -> str:

    # caption = ""
    paths = [
        f"figures/triggers/{year}/{signal}/{fom}_{channel}_minus.pdf"
        for year in ["2022", "2022EE", "2023", "2023BPix"]
    ]

    latex_text = (
        r"""
    \begin{figure}[htbp]
        \centering
        \begin{tabular}{cc}
            \includegraphics[width=0.49\textwidth]{"""
        + paths[0]
        + r"""} &
            \includegraphics[width=0.49\textwidth]{"""
        + paths[1]
        + r"""} \\[1ex]
            \includegraphics[width=0.49\textwidth]{"""
        + paths[2]
        + r"""} &
            \includegraphics[width=0.49\textwidth]{"""
        + paths[3]
        + r"""}
            \label{fig}
        \end{tabular}
    \end{figure} 
    """
    )

    return latex_text

In [ ]:
# httpt, yearsgrouped
for signal in ["bbtt", "vbfbbtt", "vbfbbtt-k2v0"]:
    # print("***",signal,"***")
    for channel in ["hh", "hm", "he"]:
        print(plot_latex_yearsgrouped(signal, channel, "httpt"))

In [ ]:
p1 = {
    "signal": ["bbtt", "bbtt", "bbtt", "bbtt"],
    "year": ["2022", "2022EE", "2023", "2023BPix"],
    "channel": ["hh", "hm", "he", "hh"],
    "fom": ["hbbpt", "hbbpt", "hbbpt", "hbbpt"],
    "isminus": [0, 0, 0, 0],
}
print(plot_latex_misc(p1))